## Identifying and resolving conflicts in a batch of VI files

Andes Reinspection 
LRG
Data information can be found at https://data.desi.lbl.gov/desi/spectro/prospect/target-reinspection-andes/

/global/cfs/cdirs/desi/spectro/redux/andes


448 visual inspections of a total of 150 unique objects

32 conflicts

In [57]:
save_filename='truth_table_Andes_reinspection_LRG_68001_20200315_4.csv'

In [58]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

import VI_utils as VI
sys.path.append("/global/homes/r/tlan/prospect/prospect/py")

from prospect import utils_specviewer,plotframes
#import desispec
# library location will change ..
sys.path.append("/global/u2/t/tlan/python/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [59]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['68001']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
#petals = ['0','3','6','7']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
#del obs_db['67230-20200314']
#obs_db.keys()

In [60]:
pd.set_option('display.max_rows', 10)

In [61]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/LRG/'

In [62]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(10,13):
    pattern = "desi*_"+str(68001)+"_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

print(vi_files)

desi*_68001_10_*.csv
desi*_68001_11_*.csv
desi*_68001_12_*.csv
['desi-vi_LRG_reinspection_68001_10_PNT.csv', 'desi-vi_LRG_reinspection_68001_10_KSD.csv', 'desi-vi_LRG_reinspection_68001_11_PNT.csv', 'desi-vi_LRG_reinspection_68001_11_KSD.csv', 'desi-vi_LRG_reinspection_68001_12_KSD.csv', 'desi-vi_LRG_reinspection_68001_12_PNT.csv']


In [63]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = ",", engine='python')
print(vi_files[0])
for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = ",", engine='python')
    vi = vi.append(vi2, ignore_index=True)
#print(vi)    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TARGETID'])

print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_LRG_reinspection_68001_10_PNT.csv
desi-vi_LRG_reinspection_68001_10_KSD.csv
desi-vi_LRG_reinspection_68001_11_PNT.csv
desi-vi_LRG_reinspection_68001_11_KSD.csv
desi-vi_LRG_reinspection_68001_12_KSD.csv
desi-vi_LRG_reinspection_68001_12_PNT.csv
There are 244 visual inspections of a total of 122 unique objects
That gives 2.0 inspections per object.


In [64]:
#vi is a dataframe
#display(vi)

In [65]:
#vi.keys()
len(vi)

244

# Merge with zbest files
#Add: fiberID, delta_chi2, flux information,.. anything else?


In [66]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

#tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_panda()    
EXPID = list(set(tf['EXPID']))[0]
tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()
print(len(tspec_df))
print(len(tf_df))
#tf_df = tf_df.rename(columns={"TargetID": "TARGETID"})
#tspec_df = tspec_df.rename(columns={"TargetID": "TARGETID"})
#print(len(vi))
vi = vi.merge(tf_df, how='left', on='TARGETID')
vi = vi.merge(tspec_df, how='left', on='TARGETID')
print(len(vi))

5000
5000
244


In [67]:
print(vi.keys())
#print(len(vi))
#vi['best redshift']=vi['VI_z']

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')


### Adding a bunch of useful columns

In [68]:
#vi['VI_z']
import VI_utils as VI
#vi['VI_z']
#bad_str = [s is None for s in vi['VI_comment']]
#vi['VI_comment'].isnull()
#bad_str = [s is None for s in vi['VI_comment']]
#print(bad_str)
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['VI_comment']
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))
#print(vi['all VI comments'])
vi = VI.prep_new_columns(vi)
#vi['best redshift'] = vi['VI_z']
#test = vi['VI_z']
#vi['best redshift'].isnull()
#vi.loc[vi['best redshift'].isnull(), 'best redshift'] = vi.loc[vi['best redshift'].isnull(), 'Redrock_z']
#vi['best redshift']

In [69]:
#check all the new columns (keys) have been added correctly
display(vi)
#print(vi.keys())

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35191284696091368,55611,20200315,68001,0.34.4,-1,-1,QSO,0.7329,PNT,...,90.962333,0.7329,GALAXY,4.0,0,0.0,0.7329,--|possibly a blend with z=0.68 galaxy; 4000 A...,2,none
1,35191284696091900,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.0359,PNT,...,147.907878,1.0359,GALAXY,4.0,0,0.0,1.0359,--|--,2,none
2,35191288219304133,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.9491,PNT,...,125.337507,0.9491,GALAXY,4.0,0,0.0,0.9491,--|--,2,none
3,35191288219304177,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.9480,PNT,...,115.874757,0.9480,GALAXY,3.5,1,0.0,0.9480,--|--,2,none
4,35191288219304552,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.6211,PNT,...,114.558437,0.6211,GALAXY,4.0,0,0.0,0.6211,--|--,2,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,35191291759297482,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.1232,PNT,...,62.786036,1.1232,GALAXY,3.5,1,0.0,1.1232,--|--,2,none
240,35191291759297857,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.0529,PNT,...,113.909822,1.0529,GALAXY,3.5,1,0.0,1.0529,--|--,2,none
241,35191291759297906,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.6336,PNT,...,42.824737,0.6336,GALAXY,3.5,1,0.0,0.6336,--|--,2,none
242,35191291763491700,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.9641,PNT,...,155.192079,0.9641,GALAXY,3.5,1,0.0,0.9641,--|--,2,none


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [70]:
vi_gp = vi.groupby(['TARGETID'])
vi_conflict = VI.find_conflicts(vi_gp)

In [71]:
len(vi_gp)

122

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [72]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35191284712866707 35191284712869404 35191284717062295 35191284717063245
 35191284729645604 35191288219305221 35191288240276813 35191291742519598
 35191291746717610 35191291750909449 35191291763492016 35191295232180973
 35191295232181877]
Total number of conflicts to resolve:  13


In [73]:
unique_targets[0]

35191284712866707

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [74]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TARGETID==unique_targets[conflict_id]][['TARGETID', 
       'Redrock_spectype', 'Redrock_z', 'VI_scanner', 'VI_quality', 'VI_issue',
       'VI_z', 'VI_spectype', 'VI_comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
    # VI interface in notebook
    if show_spectra:
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [75]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [76]:
show_spectra = False

In [77]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
20,35191284712866707,GALAXY,0.9032,PNT,4,NaN,0.9032,NaN,--,0.9032,GALAXY,2.5,3,0.9032,0.0,--|--,2,none
70,35191284712866707,GALAXY,0.9032,KSD,1,NaN,0.9032,NaN,--,0.9032,GALAXY,2.5,3,0.9032,0.0,--|--,2,none


In [78]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0885
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. very low S/N'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

20    1
70    1
Name: VI_quality, dtype: int64
conflict ID =  0


In [79]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
21,35191284712869404,GALAXY,0.9042,PNT,4,NaN,0.9042,NaN,--,0.9042,GALAXY,2.5,3,0.9042,0.0,--|--,2,none
71,35191284712869404,GALAXY,0.9042,KSD,1,NaN,0.9042,NaN,--,0.9042,GALAXY,2.5,3,0.9042,0.0,--|--,2,none


In [80]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8101
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] =1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Low S/N. Be conservative'
##look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)



conflict ID =  1


In [81]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
39,35191284717062295,GALAXY,0.7809,PNT,4,NaN,0.7809,GALAXY,--,0.7809,GALAXY,3.0,2,0.7809,0.0,--|--,2,none
89,35191284717062295,GALAXY,0.7809,KSD,2,NaN,0.7809,GALAXY,--,0.7809,GALAXY,3.0,2,0.7809,0.0,--|--,2,none


In [82]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2320
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Absorption lines may be robust. But be conservative'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  2


In [83]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
41,35191284717063245,GALAXY,0.623,PNT,4,NaN,0.623,NaN,--,0.623,GALAXY,3.0,2,0.623,0.0,--|--,2,none
91,35191284717063245,GALAXY,0.623,KSD,2,NaN,0.623,NaN,--,0.623,GALAXY,3.0,2,0.623,0.0,--|--,2,none


In [84]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5128
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Redshift should be robust'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)


conflict ID =  3


In [85]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
49,35191284729645604,GALAXY,1.6427,PNT,2,S,1.6427,NaN,low SNR Mg II is resolved and detected as an a...,1.6427,GALAXY,2.5,1,1.6427,0.374473,low SNR Mg II is resolved and detected as an a...,2,none
99,35191284729645604,GALAXY,1.6427,KSD,3,S,0.9227,NaN,break around 6650,0.9227,GALAXY,2.5,1,1.6427,0.374473,low SNR Mg II is resolved and detected as an a...,2,none


In [86]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.9227
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is robust.'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
#print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

conflict ID =  4


In [87]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
5,35191288219305221,GALAXY,1.043,PNT,4,NaN,1.0430,NaN,--,1.0430,GALAXY,3.0,2,1.043,0.059537,--|artifact mistaken for OII,2,none
55,35191288219305221,GALAXY,1.043,KSD,2,R,0.9282,NaN,artifact mistaken for OII,0.9282,GALAXY,3.0,2,1.043,0.059537,--|artifact mistaken for OII,2,none


In [88]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.9282
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Possible CaII on bad spectral region. Be conservative'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  5


In [89]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
106,35191288240276813,GALAXY,0.5651,PNT,4,NaN,0.5651,NaN,--,0.5651,GALAXY,3.0,2,0.5651,0.0,--|--,2,none
156,35191288240276813,GALAXY,0.5651,KSD,2,NaN,0.5651,NaN,--,0.5651,GALAXY,3.0,2,0.5651,0.0,--|--,2,none


In [90]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.5
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is robust. CaII and NaD'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  6


In [91]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
116,35191291742519598,GALAXY,1.6991,PNT,3,S,1.6991,NaN,bad resolution Bad subtraction at lambda=8600A...,1.6991,GALAXY,2.0,2,1.6991,0.682206,bad resolution Bad subtraction at lambda=8600A...,2,none
166,35191291742519598,GALAXY,1.6991,KSD,1,S,0.6045,NaN,--,0.6045,GALAXY,2.0,2,1.6991,0.682206,bad resolution Bad subtraction at lambda=8600A...,2,none


In [92]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.6045
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration around 8600 A. Redshift is not secure.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  7


In [93]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
128,35191291746717610,GALAXY,1.3058,PNT,3,RS,1.0000,NaN,Bad subtraction at lambda=8600A Splash in uppe...,1.0000,GALAXY,2.5,1,1.3058,0.0929,Bad subtraction at lambda=8600A Splash in uppe...,2,none
178,35191291746717610,GALAXY,1.3058,KSD,2,R,1.1858,NaN,negative OII line,1.1858,GALAXY,2.5,1,1.3058,0.0929,Bad subtraction at lambda=8600A Splash in uppe...,2,none


In [94]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.1858
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = "Bad calibration at 8600 A. Redshift is not secure."

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  8


In [95]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
132,35191291750909449,GALAXY,0.8147,PNT,3,NaN,0.8247,NaN,Ca(K and H) detected Emission lines detected a...,0.8247,GALAXY,3.0,0,0.8147,0.005511,Ca(K and H) detected Emission lines detected a...,2,none
182,35191291750909449,GALAXY,0.8147,KSD,3,NaN,0.8147,NaN,--,0.8147,GALAXY,3.0,0,0.8147,0.005511,Ca(K and H) detected Emission lines detected a...,2,none


In [96]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.8147
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift should be secure.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  9


In [97]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
221,35191291763492016,GALAXY,0.9595,KSD,2,NaN,0.9595,NaN,--,0.9595,GALAXY,3.0,2,0.9595,0.0,--|--,2,none
243,35191291763492016,GALAXY,0.9595,PNT,4,NaN,0.9595,NaN,--,0.9595,GALAXY,3.0,2,0.9595,0.0,--|--,2,none


In [98]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.6283
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. low S/N. Be conservative.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  10


In [99]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
205,35191295232180973,GALAXY,1.0391,KSD,2,NaN,1.0391,NaN,OII contaminated by telluric; real?,1.0391,GALAXY,3.0,2,1.0391,0.0,OII contaminated by telluric; real?|--,2,none
227,35191295232180973,GALAXY,1.0391,PNT,4,NaN,1.0391,NaN,--,1.0391,GALAXY,3.0,2,1.0391,0.0,OII contaminated by telluric; real?|--,2,none


In [100]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.1877
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'CaII and G band are robust.'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  11


In [101]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
208,35191295232181877,GALAXY,0.198,KSD,3,S,0.594,NaN,corrupted around 8600 AA,0.594,GALAXY,1.5,3,0.198,0.330551,corrupted around 8600 AA|Continuum detected ba...,2,none
230,35191295232181877,GALAXY,0.198,PNT,0,RCS,NaN,GALAXY,Continuum detected bad sbtraction at lambda=86...,0.198,GALAXY,1.5,3,0.198,0.330551,corrupted around 8600 AA|Continuum detected ba...,2,none


In [102]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.594
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is secure. '

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  12


### and so on...

We should now recompute the conflicts, and not find any.

In [55]:
vi_conflict = VI.find_conflicts(vi_gp)

In [103]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35191284712866707 35191284712869404 35191284717062295 35191284717063245
 35191284729645604 35191288219305221 35191288240276813 35191291742519598
 35191291746717610 35191291750909449 35191291763492016 35191295232180973
 35191295232181877]
Total number of conflicts to resolve:  13


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

Let's check the number of conflicts again.

In [104]:
vi_conflict = VI.find_conflicts(vi_gp)

In [105]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [106]:
vi['vi_combined_flag'] = vi.groupby('TARGETID')['VI_quality'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TARGETID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [107]:
vi.keys()

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2', 'best redshift',
       'best spectype', 'vi_combined_flag', 'vi_diff', 'dz', 'vi_combined_z',
       'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [108]:
print(VI_dir+save_filename)

/global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/truth_table_Andes_reinspection_LRG_68001_20200315_4.csv


In [109]:
vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',  
      'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER'].first().to_csv(VI_dir+save_filename)

<ipython-input-109-2063ed20beca>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',


Check that merged file reads in OK - check comments

In [110]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [111]:
merged_file.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'all VI comments',
       'merger comment', 'N_VI', 'DELTACHI2', 'FIBER'],
      dtype='object')

In [112]:
len(merged_file)

122

In [113]:
VI_dir

'/global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/'